In [36]:
import pandas as pd
import numpy as np
from patsy import dmatrices, dmatrix
from sklearn.linear_model import LinearRegression, Ridge, LassoLars, LassoLarsCV
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
df_train = pd.read_csv("train allstate.csv")

In [3]:
df_test = pd.read_csv("test allstate.csv")

In [89]:
print(df_train.shape)
pd.set_option('display.max_columns', 500)
df_train.head()

(188318, 132)


,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,D,B,B,D,D,B,D,C,B,D,B,A,A,A,A,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,A,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,B,D,B,D,C,B,B,B,A,A,A,A,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,D,B,B,D,D,D,B,C,B,D,B,A,A,A,A,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,C,B,B,C,A,A,A,B,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [59]:
print(df_test.shape)
df_test.head()

(125546, 131)


,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,cat40,cat41,cat42,cat43,cat44,cat45,cat46,cat47,cat48,cat49,cat50,cat51,cat52,cat53,cat54,cat55,cat56,cat57,cat58,cat59,cat60,cat61,cat62,cat63,cat64,cat65,cat66,cat67,cat68,cat69,cat70,cat71,cat72,cat73,cat74,cat75,cat76,cat77,cat78,cat79,cat80,cat81,cat82,cat83,cat84,cat85,cat86,cat87,cat88,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,B,B,C,B,D,B,A,A,A,A,A,D,C,C,E,C,D,T,H,G,A,A,G,E,I,L,K,BI,BC,A,J,AX,A,Q,HG,0.321594,0.299102,0.246911,0.402922,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,B,A,D,B,B,D,D,B,B,C,B,B,B,A,A,A,A,A,D,D,D,E,A,A,P,B,D,A,A,G,G,G,F,B,BI,CO,E,G,X,A,L,HK,0.634734,0.620805,0.654310,0.946616,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,B,A,B,B,B,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,D,B,B,B,B,B,D,C,B,B,B,A,B,A,A,A,D,D,C,E,E,A,D,G,Q,A,D,D,E,J,G,A,BI,CS,C,U,AE,A,K,CK,0.290813,0.737068,0.711159,0.412789,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,D,B,D,B,D,B,B,A,B,D,D,A,A,A,G,H,D,D,C,E,E,D,T,G,A,A,D,E,E,I,K,K,BI,CR,A,AY,AJ,A,P,DJ,0.268622,0.681761,0.592681,0.354893,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,D,B,B,D,D,B,B,C,B,B,B,A,A,A,A,A,D,B,D,E,A,A,P,A,A,A,A,F,E,G,E,B,AB,EG,A,E,I,C,J,HA,0.553846,0.299102,0.263570,0.696873,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [58]:
df_test_loss = df_test.copy()
df_test_loss["loss"] = 0
df_concat = pd.concat([df_train, df_test_loss])

In [4]:
dmat = ''
columns = df_train.columns[1:-1]

for i, c in enumerate(columns[:-1]):
    if c[:3] == 'cat':
        dmat += 'C(' + c + ') + '
    else:
        dmat += c + '+ '

dmat += columns[-1:][0]

In [32]:
y_concat, X_concat = dmatrices('loss ~ ' + dmat, df_concat, return_type="dataframe")
#X_test = dmatrix(dmat, df_concat, return_type="dataframe")
train_slice = df_train.shape[0]
y_train, X_train, X_test = y_concat[:train_slice], X_concat[:train_slice], X_concat[train_slice:]

In [5]:
y_train_init, X_train_init = dmatrices('loss ~ ' + dmat, df_train, return_type="dataframe")

In [6]:
X_test_init = dmatrix(dmat, df_test, return_type="dataframe")

In [7]:
common_columns = list(set(X_train_init.columns) & set(X_test_init.columns))

In [8]:
print(X_train_init.shape)
X_train = X_train_init.ix[:,common_columns]
print(X_train_init.shape)
print(X_train.shape)

(188318, 1038)
(188318, 1038)
(188318, 964)


In [9]:
y_train = np.ravel(y_train_init)

In [21]:
model = LassoLarsCV()
model = model.fit(X_train, y_train)

//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 50 iterations, i.e. alpha=2.558e-01, with an active set of 50 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 86 iterations, i.e. alpha=1.339e-01, with an active set of 84 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 89 iterations, i.e. alpha=1.314e-01, with an active set of 87 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/li

In [117]:
X_test = X_test_init.ix[:,common_columns]

In [119]:
y_pred = model.predict(X_test)

In [122]:
df_pred = pd.DataFrame(data={'id': df_test['id'].values, 'loss': np.ravel(y_pred)})
print(df_pred.shape)
df_pred.head()

(125546, 2)


,id,loss
0,4,1149.483055
1,6,2028.637308
2,9,11297.746321
3,12,5953.671839
4,15,235.054126


In [123]:
df_pred.to_csv('pred allstate',index=False)

In [ ]:
X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_train_init, y_train_init, test_size=0.3, random_state=0)

In [24]:
clf = LassoLarsCV()
clf = clf.fit(X_sub_train, y_sub_train)
y_sub_test_pred = clf.predict(X_sub_test)
print(mean_absolute_error(y_sub_test, y_sub_test_pred))
clf.alpha_

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 49 iterations, i.e. alpha=3.401e-01, with an active set of 49 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 91 iterations, i.e. alpha=1.701e-01, with an active set of 91 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regre

In [33]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X_sub_train, y_sub_train)
y_sub_test_pred = clf.predict(X_sub_test)
mean_absolute_error(y_sub_test, y_sub_test_pred)

1734.5801492141036

In [37]:
clf = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls')
clf = clf.fit(X_sub_train, np.ravel(y_sub_train))
y_sub_test_pred = clf.predict(X_sub_test)
mean_absolute_error(y_sub_test, y_sub_test_pred)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1411.6610355534153

## Tuning hyperparameters

In [18]:
alpha_pos = np.arange(-1.,-0.3,0.1)
alpha_pos

array([-1. , -0.9, -0.8, -0.7, -0.6, -0.5, -0.4])

In [19]:
results = {}

for a in alpha_pos:
    clf = LassoLars(alpha=a)
    clf.fit(X_sub_train, y_sub_train)
    y_sub_pred = clf.predict(X_sub_test)
    mae = mean_absolute_error(y_sub_test, y_sub_pred)
    results[a] = mae
    print("MAE with %s: %0.2f" % (a, mae))

results

//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 57 iterations, i.e. alpha=2.396e-01, with an active set of 57 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 60 iterations, i.e. alpha=2.312e-01, with an active set of 60 regressors, and the smallest cholesky pivot element being 2.107e-08
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 86 iterations, i.e. alpha=1.373e-01, with an active set of 86 regressors, and the smallest cholesky pivot element being 2.220e-16
  ConvergenceWarning)
//anaconda/lib/python3.5/site-packages/sklearn/li

MAE with -1.0: 1303.99
MAE with -0.9: 1303.99
MAE with -0.8: 1303.99
MAE with -0.7: 1303.99
MAE with -0.6: 1303.99
MAE with -0.5: 1303.99
MAE with -0.4: 1303.99


{-1.0: 1303.9851804523257,
 -0.90000000000000002: 1303.9851804523257,
 -0.80000000000000004: 1303.9851804523257,
 -0.70000000000000007: 1303.9851804523257,
 -0.60000000000000009: 1303.9851804523257,
 -0.50000000000000011: 1303.9851804523257,
 -0.40000000000000013: 1303.9851804523257}

In [ ]:
results = {}
for a in alpha_pos:
    clf = Ridge(alpha=a)
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='mean_absolute_error')
    results[a] = scores.mean()
    #scores
    print("MAE with %s: %0.2f (+/- %0.2f)" % (a, scores.mean(), scores.std() * 2))

In [ ]:
y_sub_pred = clf.predict(X_sub_test)

In [112]:
mean_absolute_error(y_sub_test, y_sub_pred)

1304.2961829972985